<a href="https://colab.research.google.com/github/AndresT3086/saberPRO-prediction/blob/main/03_modelo_svm_standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# MODELO ALTERNATIVO 1 - SVM (Support Vector Machine)
# Preprocesado: Imputación con mediana/moda + One-Hot + Normalización StandardScaler

import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [6]:
# 1. CARGAR DATOS
print('Cargando datos...')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

Cargando datos...
Train: (692500, 21)
Test: (296786, 20)


In [7]:
# 2. PREPARAR DATOS
test_ids = test['ID'].copy()
y_train = train['RENDIMIENTO_GLOBAL'].copy()
X_train = train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
X_test = test.drop(['ID'], axis=1)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')

X_train: (692500, 19)
y_train: (692500,)


In [8]:
# 3. PREPROCESADO TIPO A: Imputación + One-Hot + Normalización
print('\n=== PREPROCESADO TIPO A ===')
print('Estrategia: Mediana/Moda + One-Hot + StandardScaler')

# Identificar tipos de columnas
columnas_numericas = X_train.select_dtypes(include=['int64', 'float64']).columns
columnas_categoricas = X_train.select_dtypes(include=['object']).columns

print(f'\nNuméricas: {len(columnas_numericas)}')
print(f'Categóricas: {len(columnas_categoricas)}')


=== PREPROCESADO TIPO A ===
Estrategia: Mediana/Moda + One-Hot + StandardScaler

Numéricas: 5
Categóricas: 14


In [9]:
# 4. IMPUTAR VALORES FALTANTES
print('\nImputando valores faltantes...')

# Numéricas: rellenar con mediana
for col in columnas_numericas:
    mediana = X_train[col].median()
    X_train[col] = X_train[col].fillna(mediana)
    X_test[col] = X_test[col].fillna(mediana)

# Categóricas: rellenar con moda
for col in columnas_categoricas:
    moda = X_train[col].mode()[0]
    X_train[col] = X_train[col].fillna(moda)
    X_test[col] = X_test[col].fillna(moda)

print('Valores faltantes imputados ✓')


Imputando valores faltantes...
Valores faltantes imputados ✓


In [10]:
# 5. ONE-HOT ENCODING CON LÍMITE
print('\nAplicando One-Hot Encoding...')
print(f'Columnas antes: {X_train.shape[1]}')

# Identificar columnas categóricas con muchos valores únicos
cat_cols = X_train.select_dtypes(include=['object']).columns
print(f'\nColumnas categóricas: {len(cat_cols)}')

# Ver cuántos valores únicos tiene cada una
for col in cat_cols:
    n_unique = X_train[col].nunique()
    if n_unique > 50:
        print(f'  {col}: {n_unique} valores únicos (MUCHOS)')

# Solo hacer one-hot en columnas con menos de 20 valores únicos
cols_to_encode = []
cols_to_drop = []

for col in cat_cols:
    if X_train[col].nunique() <= 20:
        cols_to_encode.append(col)
    else:
        cols_to_drop.append(col)
        print(f'  Descartando {col} (demasiados valores: {X_train[col].nunique()})')

# Eliminar columnas con demasiados valores
X_train = X_train.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)

# One-Hot solo en las columnas seleccionadas
if len(cols_to_encode) > 0:
    X_train = pd.get_dummies(X_train, columns=cols_to_encode, drop_first=True)
    X_test = pd.get_dummies(X_test, columns=cols_to_encode, drop_first=True)
    X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print(f'\nColumnas después: {X_train.shape[1]}')
print('One-Hot Encoding completado ✓')


Aplicando One-Hot Encoding...
Columnas antes: 19

Columnas categóricas: 14
  E_PRGM_ACADEMICO: 948 valores únicos (MUCHOS)
  Descartando E_PRGM_ACADEMICO (demasiados valores: 948)
  Descartando E_PRGM_DEPARTAMENTO (demasiados valores: 31)

Columnas después: 51
One-Hot Encoding completado ✓


In [11]:
# 6. NORMALIZACIÓN CON STANDARDSCALER
print('\nNormalizando con StandardScaler...')
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print('Normalización completada ✓')


Normalizando con StandardScaler...
Normalización completada ✓


In [12]:
# 7. DIVIDIR PARA VALIDACIÓN
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42
)

print(f'\nDatos de entrenamiento: {X_train_split.shape}')
print(f'Datos de validación: {X_val_split.shape}')


Datos de entrenamiento: (554000, 51)
Datos de validación: (138500, 51)


In [13]:
# 8. ENTRENAR SGD (SVM más rápido)
print('\nEntrenando SGDClassifier (SVM rápido)...')

from sklearn.linear_model import SGDClassifier

modelo = SGDClassifier(
    loss='hinge',
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)

modelo.fit(X_train_split, y_train_split)


Entrenando SGDClassifier (SVM rápido)...


SGDClassifier(n_jobs=-1, random_state=42)

In [14]:
# 9. EVALUAR MODELO
print('\nEvaluando modelo...')
y_pred_val = modelo.predict(X_val_split)
accuracy = accuracy_score(y_val_split, y_pred_val)

print(f'\nAccuracy en validación: {accuracy:.4f} ({accuracy*100:.2f}%)')
print('\nReporte de clasificación:')
print(classification_report(y_val_split, y_pred_val))


Evaluando modelo...

Accuracy en validación: 0.3444 (34.44%)

Reporte de clasificación:
              precision    recall  f1-score   support

        alto       0.40      0.58      0.47     35165
        bajo       0.37      0.39      0.38     34573
  medio-alto       0.27      0.23      0.25     34259
  medio-bajo       0.29      0.17      0.21     34503

    accuracy                           0.34    138500
   macro avg       0.33      0.34      0.33    138500
weighted avg       0.33      0.34      0.33    138500



In [15]:
# 10. ENTRENAR MODELO FINAL
print('\nEntrenando modelo final con todos los datos...')

# Reducir también el dataset completo para el modelo final
X_train_reduced, _, y_train_reduced, _ = train_test_split(
    X_train_scaled, y_train,
    train_size=0.3,
    random_state=42,
    stratify=y_train
)

modelo_final = SGDClassifier(
    loss='hinge',
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)

modelo_final.fit(X_train_reduced, y_train_reduced)
print('Modelo final entrenado ✓')


Entrenando modelo final con todos los datos...
Modelo final entrenado ✓


In [16]:
# 11. PREDICCIONES EN TEST
print('\nHaciendo predicciones...')
predicciones = modelo_final.predict(X_test_scaled)

print(f'Predicciones realizadas: {len(predicciones)}')
print('\nDistribución de predicciones:')
print(pd.Series(predicciones).value_counts().sort_index())


Haciendo predicciones...
Predicciones realizadas: 296786

Distribución de predicciones:
alto          116018
bajo           80564
medio-alto     35245
medio-bajo     64959
Name: count, dtype: int64


In [17]:
# 12. CREAR ARCHIVO SUBMISSION
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': predicciones
})

submission.to_csv('submission_svm.csv', index=False)

print('\n✅ Archivo creado: submission_svm.csv')
print(f'Total de predicciones: {len(submission)}')
print('\nPrimeras predicciones:')
print(submission.head(10))

print('\n=== RESUMEN DEL MODELO ===')
print('Algoritmo: SVM (Support Vector Machine)')
print('Preprocesado: Mediana/Moda + One-Hot + StandardScaler')


✅ Archivo creado: submission_svm.csv
Total de predicciones: 296786

Primeras predicciones:
       ID RENDIMIENTO_GLOBAL
0  550236               alto
1   98545         medio-bajo
2  499179         medio-bajo
3  782980               bajo
4  785185               bajo
5   58495         medio-alto
6  705444               alto
7  557548               alto
8  519909         medio-bajo
9  832058               alto

=== RESUMEN DEL MODELO ===
Algoritmo: SVM (Support Vector Machine)
Preprocesado: Mediana/Moda + One-Hot + StandardScaler
